In [31]:
import requests
import json
import base64


def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Replace with your image path
test_image = "income_statement.jpg"
base64_image = encode_image(test_image)

url = "http://localhost:8001/v1/generateText"
headers = {"Content-Type": "application/json"}

data = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is the content of image? Please describe in details"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{base64_image}"
                    }
                }
            ]
        }
    ],
    "max_tokens": 1024
}

response = requests.post(url, headers=headers, data=json.dumps(data))
print("Response:", response.text)

Response: "The image is a Management Discussion and Analysis (MD&A) document comparing the financial performance of a company for two consecutive years, specifically December 31, 2022, and December 31, 2021. The document is presented in a tabular format with columns for each year and rows listing various financial metrics. Here is a detailed breakdown of the content:\n\n### Table Headers and Columns:\n- **Year ended December 31, 2022** and **2021** (RMB in millions)\n\n### Financial Metrics:\n1. **Revenue:**\n   - 2022: 280,044.0\n   - 2021: 328,309.1\n\n2. **Cost of sales:**\n   - 2022: 200,000.8\n   - 2021: 270,488.2\n\n3. **Gross profit:**\n   - 2022: 580,000.0\n   - 2021: 308,820.9\n\n4. **Research and development expenses:**\n   - 2022: 16,028.1\n   - 2021: 13,167.1\n\n5. **Selling and marketing expenses:**\n   - 2022: 21,323.3\n   - 2021: 20,980.8\n\n6. **Administrative expenses:**\n   - 2022: 5,113.9\n   - 2021: 4,738.9\n\n7. **Fair value changes on investments measured at fair 

In [32]:
# Check status code
print("Status Code:", response.status_code)  # 200 means success

# Check raw content
print("\nRaw Content:", response.content)  # Raw bytes

# Check text content
print("\nText Content:", response.text)  # String representation

Status Code: 200

Raw Content: b'"The image is a Management Discussion and Analysis (MD&A) document comparing the financial performance of a company for two consecutive years, specifically December 31, 2022, and December 31, 2021. The document is presented in a tabular format with columns for each year and rows listing various financial metrics. Here is a detailed breakdown of the content:\\n\\n### Table Headers and Columns:\\n- **Year ended December 31, 2022** and **2021** (RMB in millions)\\n\\n### Financial Metrics:\\n1. **Revenue:**\\n   - 2022: 280,044.0\\n   - 2021: 328,309.1\\n\\n2. **Cost of sales:**\\n   - 2022: 200,000.8\\n   - 2021: 270,488.2\\n\\n3. **Gross profit:**\\n   - 2022: 580,000.0\\n   - 2021: 308,820.9\\n\\n4. **Research and development expenses:**\\n   - 2022: 16,028.1\\n   - 2021: 13,167.1\\n\\n5. **Selling and marketing expenses:**\\n   - 2022: 21,323.3\\n   - 2021: 20,980.8\\n\\n6. **Administrative expenses:**\\n   - 2022: 5,113.9\\n   - 2021: 4,738.9\\n\\n7. 

In [33]:
%%writefile locustfile.py

from locust import User, task, between
import logging
import json
from PIL import Image
import base64
import time
import requests


def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

class LLMUser(User):
    
    def on_start(self):
        # Load the image once when the user starts
        self.test_image = "income_statement.jpg"
        self.base64_image = encode_image(self.test_image)
        self.url = "http://localhost:8001/v1/generateText"
        self.headers = {"Content-Type": "application/json"}
        
    @task
    def generation(self):
        data = {
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "What is the content of image? Please describe in details"
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{self.base64_image}"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 1024
        }
        
        try:
            start_time = time.time()
            response = requests.post(
                self.url,
                headers=self.headers,
                data=json.dumps(data),
                timeout=240
            )
            response_time = int((time.time() - start_time) * 1000)
            
            if response.status_code == 200:
                logging.info(f"Successful generation in {response_time}ms")
                print("Response:", response.text)
                self.environment.events.request.fire(
                    request_type="POST",
                    name="Generate Text",
                    response_time=response_time,
                    response_length=len(response.text),
                    response=response,
                    context={}
                )
            else:
                logging.error(f"Failed with status code: {response.status_code}")
                self.environment.events.request.fire(
                    request_type="POST",
                    name="Generate Text",
                    response_time=response_time,
                    response_length=len(response.text),
                    response=response,
                    context={},
                    exception=f"HTTP {response.status_code}"
                )
                
        except Exception as e:
            logging.error(f"Generation failed: {str(e)}")
            self.environment.events.request.fire(
                request_type="POST",
                name="Generate Text",
                response_time=0,
                response_length=0,
                exception=e,
                context={}
            )
            time.sleep(1)  # Add small delay after errors

Overwriting locustfile.py


In [34]:
!locust --headless --users 4 --spawn-rate 4 --run-time 120 --csv ./benchmark_metric/benchmark_u30

[2024-11-12 02:54:56,409] ip-172-16-164-122/INFO/locust.main: Starting Locust 2.32.1
[2024-11-12 02:54:56,409] ip-172-16-164-122/INFO/locust.main: Run time limit set to 120 seconds
Type     Name  # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated       0     0(0.00%) |      0       0       0      0 |    0.00        0.00

[2024-11-12 02:54:56,410] ip-172-16-164-122/INFO/locust.runners: Ramping to 4 users at a rate of 4.00 per second
[2024-11-12 02:54:56,411] ip-172-16-164-122/INFO/locust.runners: All users spawned: {"LLMUser": 4} (4 total users)
Type     Name  # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
--------||-------|-------------|-------|-------|-------|------